In [3]:
import sys
if ".." not in sys.path:
    sys.path.append("..")
import openpyxl
from energydata import energinet

import importlib
energinet = importlib.reload(energinet)

In [4]:
descriptor = {"resource_id": "connectionpointsingrid"}
service = energinet.EnergiDataServiceDk(descriptor)

In [5]:
list(energinet.fetch_edh(descriptor, 1))

[{'help': 'https://api.energidataservice.dk/help_show?name=datastore_search',
  'success': True,
  'result': {'include_total': True,
   'resource_id': '2feb8df3-d3c3-4586-a485-19782797cc45',
   'fields': [{'type': 'int', 'id': '_id'},
    {'info': {'notes': 'Abbreviation for stations in the electricity grid.'},
     'type': 'text',
     'id': 'ConnectionPointCode'},
    {'info': {'notes': 'Long name for stations in the electricity grid.'},
     'type': 'text',
     'id': 'ConnectionPointName'},
    {'info': {'notes': 'Each of the 98 Danish municipalities has a unique number, ranging from 101 Copenhagen to 860 Hjørring.'},
     'type': 'text',
     'id': 'MunicipalityNo'},
    {'info': {'notes': 'Name of municipality  in Denmark'},
     'type': 'text',
     'id': 'Municipality'},
    {'info': {'notes': 'Administrative regions in Denmark'},
     'type': 'text',
     'id': 'RegionName'},
    {'info': {'notes': 'EPSG:4326'},
     'type': 'float8',
     'id': 'ConnectionPointLatitude'},
   

In [6]:
list(service.fetch_data())

[{'_id': 1044,
  'ConnectionPointCode': 'ALMV',
  'ConnectionPointName': 'Alminde',
  'MunicipalityNo': '621',
  'Municipality': 'Kolding',
  'RegionName': 'Syddanmark',
  'ConnectionPointLatitude': 55.572477,
  'ConnectionPointLongitude': 9.48877,
  'NetCompanyName': 'TREFOR',
  'region_code': 'dk_mun_621'},
 {'_id': 20,
  'ConnectionPointCode': 'AMK',
  'ConnectionPointName': 'Amager koblingsstation',
  'MunicipalityNo': '101',
  'Municipality': 'København',
  'RegionName': 'Hovedstaden',
  'ConnectionPointLatitude': 55.641284,
  'ConnectionPointLongitude': 12.609259,
  'NetCompanyName': 'Radius Elnet',
  'region_code': 'dk_mun_101'},
 {'_id': 22,
  'ConnectionPointCode': 'AMV',
  'ConnectionPointName': 'Amagerværket',
  'MunicipalityNo': '101',
  'Municipality': 'København',
  'RegionName': 'Hovedstaden',
  'ConnectionPointLatitude': 55.685241,
  'ConnectionPointLongitude': 12.624494,
  'NetCompanyName': 'Radius Elnet',
  'region_code': 'dk_mun_101'},
 {'_id': 23,
  'ConnectionPoint

In [7]:
metadata = service.fetch_metadata()
data = service.fetch_data()

In [8]:
connection_points = {
    row["ConnectionPointCode"] : row for row in data
}

In [10]:
# ws = openpyxl.load_workbook("../Energinet Extra Codes.xlsx").active

# [column_names, *rows] = list(ws.values)
# extra_station_codes = {}
# for row in rows:
#     d = dict(zip(column_names, row))
#     if d["Code"] is not None:
#         extra_station_codes[d["Name"]] = d["Code"]

In [11]:
# extra_station_codes

In [76]:
extra_station_codes = {
    'A2': 'BJS',
    'AMF_132_F': 'AMV',
    'HASØ132_F': 'HAS',
    'ISHN132_N-$': 'ISH',
    'MRP1': 'TEG',
    'VSAK132_S1': 'VAL',
    'BIX_150_S1': 'BIL',
    'FRX_150_S1': 'FRT',
    'I4': 'ABS',
    'KLT_150_S1': 'KLF',
    'VKHI012_SFIK': 'VKE',
    'term_KT51-HT3': 'TJE',
}

In [13]:
import io
import urllib.request

In [14]:
netdata_url = "https://energinet.dk/-/media/76C3D434A7E2499F94E7DB66D3A2DE21.xlsx?la=da&hash=638E165AA57F627409CB9394D56D53614F5D7336"

with urllib.request.urlopen(netdata_url) as fileobj:
    buf = io.BytesIO()
    buf.write(fileobj.read())

buf.seek(0)
wb = openpyxl.load_workbook(buf)

In [15]:
wb.sheetnames

['Bus',
 'Line',
 'Transformer2',
 'Transformer3',
 'Generator',
 'Load',
 'Shunt',
 'HVDC',
 'PFModelInfo',
 'Ark1',
 'Ark2',
 'Ark3']

In [19]:
import networkx as nx

In [20]:
def get_sheet_data(ws):
    [_1, _2, _3, header, *rows] = ws.values
    for row in rows:
        yield {k: v for k, v in zip(header, row)}

In [109]:
import re
def match_bus_to_code(d, codes, extra_codes):
    extensions = {
        "ENDKW": "V",
        "ENDKE": "Ø"
    }
    name = d["Bus Name"]
    if name in extra_codes and extra_codes[name] in codes:
        return extra_codes[name]
    name_regex = re.compile(r"([^0-9_]+)_?(\d*)_?(.*)")
    match = name_regex.match(name)
    if not match:
        return
    bus_code = match.groups()[0]
    if bus_code in codes:
        return bus_code
    star_codes = [f"{bus_code}*", f"{bus_code}**", f"{bus_code}***"]
    for star_code in star_codes:
        if star_code in codes:
            return star_code
    if d["Location Name"] in extensions:
        area_code = bus_code + extensions[d["Location Name"]]
        if area_code in codes:
            return area_code

In [110]:
energy_grid = nx.Graph()

In [111]:
bus_labels = {}
for d in get_sheet_data(wb["Bus"]):
    node_label = ("Bus", d["Bus Name"])
    bus_labels[d["Bus Index"]] = node_label
    attributes = {k: v for k, v in d.items()}
    connection_code = match_bus_to_code(d, list(connection_points), extra_station_codes)
    if connection_code is not None:
        attributes.update(connection_points[connection_code])
    energy_grid.add_nodes_from([(node_label, attributes)])

In [112]:
for d in get_sheet_data(wb["Line"]):
    energy_grid.add_edges_from([(bus_labels[d["Node 1"]], bus_labels[d["Node 2"]], d)])

In [113]:
for d in get_sheet_data(wb["Transformer2"]):
    node_label = ("Transformer3", d["2-Transformer Name"])    
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edges_from([
        (bus_labels[d["High.V Bus Index"]], node_label), 
        (bus_labels[d["Low.V Bus Index"]], node_label)
    ])

In [114]:
for d in get_sheet_data(wb["Transformer3"]):
    node_label = ("Transformer3", d["3_Transformer Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    edges = [
        (bus_labels[d["High.V Bus Index"]], node_label),
        (bus_labels[d["Mid.V Bus Index"]], node_label),        
        (bus_labels[d["Low.V Bus Index"]], node_label)
    ]
    energy_grid.add_edges_from(edges)

In [115]:
for d in get_sheet_data(wb["Generator"]):
    node_label = ("Generator", d["Generator Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edge(node_label, bus_labels[d["Bus Index"]])

In [116]:
for d in get_sheet_data(wb["Load"]):
    node_label = ("Load", d["Load Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edge(node_label, bus_labels[d["Load Index"]])

In [117]:
for d in get_sheet_data(wb["Shunt"]):
    node_label = ("Shunt", d["Shunt Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edge(node_label, bus_labels[d["Bus Index"]])

In [118]:
for d in get_sheet_data(wb["HVDC"]):
    node_label = ("HVDC", d["HVDC Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edge(node_label, bus_labels[d["Bus Index"]])

In [119]:
[len(x) for x in nx.connected_components(energy_grid)]

[451, 561, 1, 1]

In [120]:
[x for x in nx.connected_components(energy_grid) if len(x) == 1]

[{('Bus', 'EDR_400_NIE1')}, {('Bus', 'EDR_400_NIE1(1)')}]

In [127]:
[(n, v) for n, v in energy_grid.nodes.items() if n[0] == "Bus" and "region_code" not in v and v["Location Name"] in ["ENDKE", "ENDKW"]]

[(('Bus', 'AHA_220_S1'),
  {'Bus Index': 195,
   'Bus Name': 'AHA_220_S1',
   'Station Full Name': 'Anholt Havmøllepark',
   'Location Name': 'ENDKW',
   'Voltage base[kV]': '232,0',
   'Voltage min[pu]': '0,98',
   'Voltage max[pu]': '1,02'}),
 (('Bus', 'DDS_1321_SFIK'),
  {'Bus Index': 85,
   'Bus Name': 'DDS_1321_SFIK',
   'Station Full Name': '132 KV STATION STÅLVALSEVÆRKET',
   'Location Name': 'ENDKE',
   'Voltage base[kV]': '132,0',
   'Voltage min[pu]': '0,95',
   'Voltage max[pu]': '1,05'}),
 (('Bus', 'DDS_1322_SFIK'),
  {'Bus Index': 86,
   'Bus Name': 'DDS_1322_SFIK',
   'Station Full Name': '132 KV STATION STÅLVALSEVÆRKET',
   'Location Name': 'ENDKE',
   'Voltage base[kV]': '132,0',
   'Voltage min[pu]': '0,95',
   'Voltage max[pu]': '1,05'}),
 (('Bus', 'FEM_132_F'),
  {'Bus Index': 90,
   'Bus Name': 'FEM_132_F',
   'Station Full Name': '132 KV STATION FEMERN',
   'Location Name': 'ENDKE',
   'Voltage base[kV]': '132,0',
   'Voltage min[pu]': '0,00',
   'Voltage max[pu]':

In [134]:
{k: v for k, v in connection_points.items() if "skuder" in v["ConnectionPointName"].lower()}

{}

In [139]:
g0 = energy_grid
g = g0.copy()
# split_nodes = [n for n, data in g0.nodes.items() if "region_code" in data]
split_nodes = [n for n, data in g0.nodes.items() if n[0] == "Bus"]
split_mapping = {}
for n in split_nodes:
    edges = g[n]
    data = g.nodes[n]
    g.remove_node(n)
    g.remove_edges_from(n)
    for i, (neighbour, edge_data) in enumerate(edges.items()):
        split_node = (i, n)
        g.add_nodes_from([(split_node, data)])
        g.add_edges_from([(split_node, neighbour, edge_data)])
        split_mapping[split_node] = n

In [140]:
g_ccs = [g.subgraph(cc) for cc in nx.connected_components(g) if len(cc) > 2]
g_ccs = sorted(g_ccs, key=lambda x: -len(x))
len(g_ccs)

70

In [148]:
extra_connections = [set(split_mapping[k] for k, v in cc.nodes.items() if k in split_mapping) for cc in g_ccs]
extra_edges = set(tuple(sorted((a, b))) for x in extra_connections for a in x for b in x if a != b)
extra_edges

{(('Bus', 'A2'), ('Bus', 'BJS_220_SFIK')),
 (('Bus', 'A2'), ('Bus', 'BJS_400_K')),
 (('Bus', 'ABS_150_S2-'), ('Bus', 'I4')),
 (('Bus', 'AMV_010_SFIK'), ('Bus', 'AMV_132_A')),
 (('Bus', 'AMV_0173_SFIK'), ('Bus', 'AMV_132_A')),
 (('Bus', 'ASR_150_S2'), ('Bus', 'ASR_400_S1')),
 (('Bus', 'ASV_0122_SFIK'), ('Bus', 'ASV_132_F')),
 (('Bus', 'ASV_0215_SFIK'), ('Bus', 'ASV_400_K')),
 (('Bus', 'ASV_132_F'), ('Bus', 'ASV_4001_SFIK')),
 (('Bus', 'ASV_4001_SFIK'), ('Bus', 'ASV_400_K')),
 (('Bus', 'AVV_0122_SFIK'), ('Bus', 'AVV_0123_SFIK')),
 (('Bus', 'AVV_0122_SFIK'), ('Bus', 'AVV_400_SK.K')),
 (('Bus', 'AVV_0123_SFIK'), ('Bus', 'AVV_400_SK.K')),
 (('Bus', 'AVV_0171_SFIK'), ('Bus', 'AVV_132_F')),
 (('Bus', 'AVV_0201_SFIK'), ('Bus', 'AVV_400_SK.K')),
 (('Bus', 'AVV_132_F'), ('Bus', 'AVV_132_SFIK')),
 (('Bus', 'AVV_132_SFIK'), ('Bus', 'AVV_400_SK.K')),
 (('Bus', 'BJS_016'), ('Bus', 'BJS_400_K')),
 (('Bus', 'BJS_132_SFIK'), ('Bus', 'BJS_400_K')),
 (('Bus', 'EBY'), ('Bus', 'LIN_132_F')),
 (('Bus', 'EDR

In [131]:
g2 = g_ccs[0]
layout = nx.layout.kamada_kawai_layout(g2)

In [132]:
import plotly.graph_objs as go
import numpy as np

traces = []
lines = []
points = []
node_keys = list(layout.keys())
for k in node_keys:
    points.append(layout[k])
for n1, n2 in g2.edges:
    lines.append(layout[n1])
    lines.append(layout[n2])
    lines.append([np.nan, np.nan])


traces.append({"x": [p[0] for p in lines], "y": [p[1] for p in lines],  "mode": "lines"})
traces.append({"x": [p[0] for p in points], "y": [p[1] for p in points], "text": node_keys, "mode": "markers"})
go.Figure(data=traces).show()

In [149]:
energy_grid[next(iter(energy_grid.nodes()))]

AtlasView({('Transformer3', 'BJS_220_2201-T31'): {}, ('Transformer3', 'BJS_400_220-T43'): {}})

In [151]:
cross_municipality_lines = []
for n1, n2 in list(energy_grid.edges) + list(extra_edges):
    try:
        r1 = energy_grid.nodes[n1]["region_code"]
        r2 = energy_grid.nodes[n2]["region_code"]
    except KeyError:
        continue
    if r1 != r2:
        cross_municipality_lines.append((n1, n2))
len(cross_municipality_lines)

146

In [152]:
def node_point(node):
    x = energy_grid.nodes[node]
    return (x["ConnectionPointLatitude"], x["ConnectionPointLongitude"])

connection_lines = [(node_point(n1), node_point(n2)) for n1, n2 in cross_municipality_lines]

import json
with open("connection_points.json", "w+") as fileobj:
    json.dump(connection_lines, fileobj)

In [ ]:
municipality_nodes = {k: v["region_code"] for k, v in energy_grid.nodes.items() if "region_code" in v}
municipality_nodes

In [90]:
municipality_connections = set()
for n1, n2 in energy_grid.edges:
    try:
        m1 = municipality_nodes[n1]
        m2 = municipality_nodes[n2]
        if m2 < m1:
            m1, m2 = m2, m1
        municipality_connections.add((m1, m2))
    except KeyError:
        pass
municipality_connections

{('dk_mun_101', 'dk_mun_101'),
 ('dk_mun_101', 'dk_mun_159'),
 ('dk_mun_101', 'dk_mun_167'),
 ('dk_mun_101', 'dk_mun_185'),
 ('dk_mun_151', 'dk_mun_240'),
 ('dk_mun_159', 'dk_mun_159'),
 ('dk_mun_159', 'dk_mun_161'),
 ('dk_mun_159', 'dk_mun_240'),
 ('dk_mun_161', 'dk_mun_161'),
 ('dk_mun_167', 'dk_mun_183'),
 ('dk_mun_169', 'dk_mun_183'),
 ('dk_mun_169', 'dk_mun_240'),
 ('dk_mun_169', 'dk_mun_265'),
 ('dk_mun_169', 'dk_mun_350'),
 ('dk_mun_183', 'dk_mun_240'),
 ('dk_mun_183', 'dk_mun_259'),
 ('dk_mun_183', 'dk_mun_265'),
 ('dk_mun_201', 'dk_mun_217'),
 ('dk_mun_201', 'dk_mun_219'),
 ('dk_mun_217', 'dk_mun_219'),
 ('dk_mun_217', 'dk_mun_260'),
 ('dk_mun_219', 'dk_mun_219'),
 ('dk_mun_219', 'dk_mun_240'),
 ('dk_mun_219', 'dk_mun_260'),
 ('dk_mun_240', 'dk_mun_240'),
 ('dk_mun_240', 'dk_mun_250'),
 ('dk_mun_240', 'dk_mun_259'),
 ('dk_mun_250', 'dk_mun_250'),
 ('dk_mun_250', 'dk_mun_350'),
 ('dk_mun_253', 'dk_mun_269'),
 ('dk_mun_260', 'dk_mun_260'),
 ('dk_mun_265', 'dk_mun_265'),
 ('dk_mu

In [91]:
energy_grid.nodes[list(municipality_nodes)[0]]

{'Bus Index': 76,
 'Bus Name': 'A2',
 'Station Full Name': 'Bjæverskov',
 'Location Name': 'ENDKE',
 'Voltage base[kV]': '232,0',
 'Voltage min[pu]': '0,00',
 'Voltage max[pu]': '1,05',
 '_id': 76,
 'ConnectionPointCode': 'BJS',
 'ConnectionPointName': 'Bjæverskov',
 'MunicipalityNo': '259',
 'Municipality': 'Køge',
 'RegionName': 'Sjælland',
 'ConnectionPointLatitude': 55.45053,
 'ConnectionPointLongitude': 12.00791,
 'NetCompanyName': 'SEAS/NVE TRANSMISSION',
 'region_code': 'dk_mun_259'}

In [92]:
import sqltables
db = sqltables.Database()
latlon = db.create_table(column_names=["region_code", "lat", "lon"])
latlon.insert([[x["region_code"], x["ConnectionPointLatitude"], x["ConnectionPointLongitude"]] 
               for x in (energy_grid.nodes[k] for k in municipality_nodes.keys())])

In [93]:
mun_latlon = latlon.table("""select region_code, avg(lat) as lat, avg(lon) as lon from _ group by region_code""")
mun_points = {row.region_code: (row.lat, row.lon) for row in mun_latlon}

In [94]:
mun_points

{'dk_mun_101': (55.67414544444444, 12.590287444444444),
 'dk_mun_151': (55.744171, 12.32608),
 'dk_mun_159': (55.7501205, 12.485704),
 'dk_mun_161': (55.686573, 12.397478),
 'dk_mun_167': (55.604988999999996, 12.480812),
 'dk_mun_169': (55.660772, 12.319345),
 'dk_mun_183': (55.6346, 12.32171),
 'dk_mun_185': (55.631542, 12.657965),
 'dk_mun_201': (55.885227, 12.338456),
 'dk_mun_217': (56.028681, 12.563658),
 'dk_mun_219': (55.8955735, 12.196541750000002),
 'dk_mun_240': (55.755691, 12.198973),
 'dk_mun_250': (55.80958483333333, 11.896245666666665),
 'dk_mun_253': (55.585099, 12.258356),
 'dk_mun_259': (55.45053, 12.00791),
 'dk_mun_260': (55.961136, 12.019283),
 'dk_mun_265': (55.6285435, 12.1149735),
 'dk_mun_269': (55.54376, 12.20485),
 'dk_mun_306': (55.871017, 11.603077),
 'dk_mun_316': (55.717711, 11.491859),
 'dk_mun_326': (55.66188011111111, 11.087549777777777),
 'dk_mun_340': (55.545725, 11.641376),
 'dk_mun_350': (55.645783333333334, 11.932815666666668),
 'dk_mun_360': (54.8

In [95]:
connection_points = {(m1, m2): (mun_points[m1], mun_points[m2]) for m1, m2 in municipality_connections}

import json
with open("connection_points.json", "w+") as fileobj:
    json.dump(list(connection_points.values()), fileobj)